In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet50,resnet18
import torch
import torchvision
from torchvision.models import resnet50
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
from collections import OrderedDict
import torchsummary
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms, models

In [ ]:
class Activation():
    def __init__(self,module):
        self.hook = module.register_forward_hook(self.hook_fn)

    def hook_fn(self, module, input, output):
        self.features = output.detach()

    def remove(self):
        self.hook.remove()

In [ ]:
def layer_list(model):
    list = OrderedDict()
    for name,layer in model.named_modules():
        if name == '':
            continue
        list[name] = layer
    return list

# main

In [ ]:
model = resnet18()
model.eval()

In [ ]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
        #activation[name] = output.data.numpy()
    return hook

In [ ]:
for name,module in model.named_modules():
    if isinstance(module,nn.Conv2d):
        module.register_forward_hook(get_activation(name))

In [ ]:
activation = {}
activation

In [ ]:
output = model(torch.randn(3,224,224).unsqueeze(0))

In [ ]:
activation

In [ ]:
torchsummary.summary(model,(3,224,224), device = 'cpu')

# 이부분

In [81]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import os
import numpy as np
import pandas as pd
import sys
from models.resnet import resnet18

use_layer = ['layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv2', 'layer3.0.conv1',
            'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv1', 'layer4.1.conv2']

def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
        #activation[name] = output.data.numpy()
    return hook

save_path = r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\test\actMap'
image_path = r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\test\images'
state_path = r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\results\experiment\baseline\99_net_sign.pth'
# ResNet-18 모델을 불러옵니다.
model = resnet18()
#model = torch.nn.Sequential(*list(model.children())[:-3])
model.load_state_dict(torch.load(state_path))
loss = torch.nn.MSELoss()

In [82]:
model.eval()
for name,module in model.named_modules():
    #print(module)
    if isinstance(module,nn.Conv2d):
        module.register_forward_hook(get_activation(name))


# 이미지 변환을 위한 전처리 함수를 정의합니다.
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])


for img in os.listdir(image_path):
    print(img)
    # 이미지를 불러옵니다.
    image = Image.open(os.path.join(image_path,img)).convert("RGB")
    dir_path = os.path.join(save_path,img)
    
    if os.path.isdir(dir_path) == False:
        os.mkdir(dir_path)
    # 이미지를 전처리합니다.
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)
    x = input_batch
    # 이미지를 모델에 통과시켜 activation map을 얻습니다.
    
    activation = {}
    output = model(x)
    
    # 각 레이어의 activation map을 파일로 저장합니다.
    for idx, activation_map in enumerate(activation.items()):
        path = os.path.join(save_path,img)
        file_name = f'{img}_{activation_map[0]}'
        to_np = activation_map[1].numpy()
        np.save(os.path.join(path,file_name),activation_map[1])
        #print(f"Activation map for Layer {idx+1}({activation_map[0]}): {activation_map[1].shape} saved")
        #activation_map = activation_map.squeeze(dim=0)
    #print(f"{img}'s npy file saved")

27-양화로 상3 근_002.jpg
마포대교 신규등록3-general-0_004.jpg


In [83]:
np_path = r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\test\actMap'
npList = os.listdir(np_path)
npList

['27-양화로 상3 근_002.jpg', '마포대교 신규등록3-general-0_004.jpg']

In [84]:
for i in activation.items():
    print(i[0])

conv1
layer1.0.conv1
layer1.0.conv2
layer1.1.conv1
layer1.1.conv2
layer2.0.conv1
layer2.0.conv2
layer2.0.downsample.0
layer2.1.conv1
layer2.1.conv2
layer3.0.conv1
layer3.0.conv2
layer3.0.downsample.0
layer3.1.conv1
layer3.1.conv2
layer4.0.conv1
layer4.0.conv2
layer4.0.downsample.0
layer4.1.conv1
layer4.1.conv2


In [68]:
#같은 class
for ori in npList:
    mse = 0
    map1 = os.path.join(np_path,ori)
    for comp in npList:
        mse = 0
        #min_loss = sys.maxsize
        #min_loss_conv = ''
        map2 = os.path.join(np_path,comp)
        if ori == comp:
            continue
        for activation_map in activation.items():
            n1 = torch.from_numpy(np.load(os.path.join(map1,ori) + f'_{activation_map[0]}.npy'))
            n2 = torch.from_numpy(np.load(os.path.join(map2,comp) + f'_{activation_map[0]}.npy'))
            #result = np.mean((n1-n2)**2)
            result = loss(n1,n2)
            #mse += result
            print(f"{ori} and {comp}'s {activation_map[0]} loss : {result}")
        print(f"{ori} and {comp}'s loss sum : {mse}")
        break
    break

마포대교 신규등록3-general-0_004.jpg and 용산효창원길0013(근)-general-0_004.jpg's conv1 loss : 0.27670812606811523
마포대교 신규등록3-general-0_004.jpg and 용산효창원길0013(근)-general-0_004.jpg's layer1.0.conv1 loss : 6.959519863128662
마포대교 신규등록3-general-0_004.jpg and 용산효창원길0013(근)-general-0_004.jpg's layer1.0.conv2 loss : 8.227204322814941
마포대교 신규등록3-general-0_004.jpg and 용산효창원길0013(근)-general-0_004.jpg's layer1.1.conv1 loss : 23.035491943359375
마포대교 신규등록3-general-0_004.jpg and 용산효창원길0013(근)-general-0_004.jpg's layer1.1.conv2 loss : 23.595420837402344
마포대교 신규등록3-general-0_004.jpg and 용산효창원길0013(근)-general-0_004.jpg's layer2.0.conv1 loss : 35.7568244934082
마포대교 신규등록3-general-0_004.jpg and 용산효창원길0013(근)-general-0_004.jpg's layer2.0.conv2 loss : 42.86428451538086
마포대교 신규등록3-general-0_004.jpg and 용산효창원길0013(근)-general-0_004.jpg's layer2.0.downsample.0 loss : 2.043657064437866
마포대교 신규등록3-general-0_004.jpg and 용산효창원길0013(근)-general-0_004.jpg's layer2.1.conv1 loss : 58.09381866455078
마포대교 신규등록3-general-0_004.jpg and 용산효

In [85]:
#다른 class
for ori in npList:
    mse = 0
    map1 = os.path.join(np_path,ori)
    for comp in npList:
        mse = 0
        #min_loss = sys.maxsize
        #min_loss_conv = ''
        map2 = os.path.join(np_path,comp)
        if ori == comp:
            continue
        for activation_map in activation.items():
            n1 = torch.from_numpy(np.load(os.path.join(map1,ori) + f'_{activation_map[0]}.npy'))
            n2 = torch.from_numpy(np.load(os.path.join(map2,comp) + f'_{activation_map[0]}.npy'))
            #result = np.mean((n1-n2)**2)
            result = loss(n1,n2)
            mse += result
            print(f"{ori} and {comp}'s {activation_map[0]} loss : {result}")
        print(f"{ori} and {comp}'s loss sum : {mse}")
        break
    break

27-양화로 상3 근_002.jpg and 마포대교 신규등록3-general-0_004.jpg's conv1 loss : 0.9034370183944702
27-양화로 상3 근_002.jpg and 마포대교 신규등록3-general-0_004.jpg's layer1.0.conv1 loss : 20.517311096191406
27-양화로 상3 근_002.jpg and 마포대교 신규등록3-general-0_004.jpg's layer1.0.conv2 loss : 20.578065872192383
27-양화로 상3 근_002.jpg and 마포대교 신규등록3-general-0_004.jpg's layer1.1.conv1 loss : 48.40366744995117
27-양화로 상3 근_002.jpg and 마포대교 신규등록3-general-0_004.jpg's layer1.1.conv2 loss : 42.94108200073242
27-양화로 상3 근_002.jpg and 마포대교 신규등록3-general-0_004.jpg's layer2.0.conv1 loss : 60.58995819091797
27-양화로 상3 근_002.jpg and 마포대교 신규등록3-general-0_004.jpg's layer2.0.conv2 loss : 79.87269592285156
27-양화로 상3 근_002.jpg and 마포대교 신규등록3-general-0_004.jpg's layer2.0.downsample.0 loss : 3.653759479522705
27-양화로 상3 근_002.jpg and 마포대교 신규등록3-general-0_004.jpg's layer2.1.conv1 loss : 79.75484466552734
27-양화로 상3 근_002.jpg and 마포대교 신규등록3-general-0_004.jpg's layer2.1.conv2 loss : 51.03647232055664
27-양화로 상3 근_002.jpg and 마포대교 신규등록3-general-0_004.

# train test

In [ ]:
model = resnet18()

In [61]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import os
import numpy as np
import pandas as pd
import sys
from models.resnet import resnet18

use_layer = ['layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv2', 'layer3.0.conv1',
            'layer3.0.conv2', 'layer4.0.conv1', 'layer4.1.conv1', 'layer4.1.conv2']

def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
        #activation[name] = output.data.numpy()
    return hook

loss_criterion = nn.MSELoss()
save_path = r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\test\actMap'
image_path = r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\test\images'
state_path = r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\results\experiment\baseline\99_net_sign.pth'
# ResNet-18 모델을 불러옵니다.
model = resnet18()
#model = torch.nn.Sequential(*list(model.children())[:-3])
model.load_state_dict(torch.load(state_path))

<All keys matched successfully>

In [62]:
model.eval()
for name,module in model.named_modules():
    #print(module)
    #if isinstance(module,nn.Conv2d):
    if name in use_layer:
        module.register_forward_hook(get_activation(name))
# 이미지를 불러오고 전처리하는 변환 함수
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

activation = {}
# 이미지들을 묶어서 처리할 배치 텐서 생성
batch = []
for img in os.listdir(image_path):
    image = Image.open(os.path.join(image_path,img)).convert("RGB")
    input_tensor = preprocess(image)
    batch.append(input_tensor)

# 배치 텐서 생성
batch_tensor = torch.stack(batch, dim=0)

# 배치 텐서를 모델에 전달하여 activation 얻기
with torch.no_grad():
    output = model(batch_tensor)

print("Activation shape:", output.shape)

Activation shape: torch.Size([2, 569])


In [63]:
for a in activation.items():
    print(a[0])

layer1.1.conv1
layer1.1.conv2
layer2.0.conv1
layer2.0.conv2
layer2.1.conv2
layer3.0.conv1
layer3.0.conv2
layer4.0.conv1
layer4.1.conv1
layer4.1.conv2


In [ ]:
for a in activation.items():
    #print(a[1][0].shape)
    #temp = a[1][0].unsqueeze(0)
    #print(temp.shape)
    #print(a[1].shape)
    print(a[1][0].unsqueeze(0).shape)

In [ ]:
model.eval()
for name,module in model.named_modules():
    #print(module)
    if isinstance(module,nn.Conv2d):
        module.register_forward_hook(get_activation(name))


# 이미지 변환을 위한 전처리 함수를 정의합니다.
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])


for img in os.listdir(image_path):
    # 이미지를 불러옵니다.
    #image = Image.open(r'D:\ms\capstone\dataset\custom\comb\test_for_actMap\general2.jpg').convert("RGB")
    image = Image.open(os.path.join(image_path,img)).convert("RGB")
    print(image)
    # 이미지를 전처리합니다.
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)
    x = input_batch
    # 이미지를 모델에 통과시켜 activation map을 얻습니다.
    
    activation = {}
    output = model(x)

In [ ]:
activation['conv1']

In [ ]:
activation['conv1'].shape

In [ ]:
print(np.load(r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\activationMap\0\0_conv1.npy').shape)
print(np.load(r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\activationMap\0\0_layer1.0.conv1.npy').shape)
print(np.load(r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\activationMap\0\0_layer2.1.conv2.npy').shape)
print(np.load(r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\activationMap\0\0_layer4.1.conv1.npy').shape)
print(np.load(r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\activationMap\0\0_layer4.1.conv2.npy').shape)

In [ ]:
def act_iter(activation):
    for a in activation:
        yield a

In [ ]:
def comb_act(activation):
    yield from act_iter(activation)

In [ ]:
print(torch.from_numpy(np.load(r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\activationMap\1\1_conv1.npy')))

In [ ]:
def optimize_feature(activation,label):
    path = os.path.join(act_path,label)
    for a in activation.items():
        file_name = os.path.join(path,f'{label}_{a[0]}.npy')
        print(file_name)
        sub_param = nn.Parameter(torch.from_numpy(np.load(file_name)))
        optimizer = optim.Adam([sub_param],lr=0.01)
        loss = loss_criterion(sub_param,a[1])
        #print(loss)
        loss.backward()
        optimizer.step()
        #print(type(sub_param.data))
        #np.save(file_name,sub_param.data.numpy())
        #print(sub_param.data)

In [ ]:
optimize_feature(activation,str(1))

In [ ]:
optimize_feature(activation,str(1))

In [ ]:
t_path = r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\test_map\1'

In [ ]:
for d in os.listdir(t_path):
    #print(d)
    print(np.load(os.path.join(t_path,d)))

In [ ]:
for d in os.listdir(t_path):
    #print(d)
    print(np.load(os.path.join(t_path,d)))

# optimizer

In [ ]:
n2 = torch.from_numpy(np.load(r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\test_map\0\0_conv1.npy'))

In [ ]:
sub_param = nn.Parameter(n2)
#sub_param = nn.Parameter(torch.Tensor(n2))
#optimizer = optim.SGD([sub_param], lr=0.1)
optimizer = optim.Adam([sub_param],lr=0.01)
loss_criterion = nn.MSELoss()

for epoch in range(5):
    optimizer.zero_grad() 
    
    #loss.requires_grad_(True)
    loss = loss_criterion(sub_param,n1)
    print(loss)
    

    loss.backward()
    optimizer.step()  
    
    #print(sub_param.data)
    #print("n2:")
    #print(n2)

# 행렬 합치기

In [ ]:
arrays = []
for i in activation.items():
    arrays.append(i[1])

In [ ]:
arrays[0].shape

In [ ]:
print('shapes start')
shapes = [n.shape for n in arrays] 
print('shapes done')

print('final start')
final = np.vstack(shapes).max(axis=0)
print('final done')

print('out start')

out = np.concatenate([np.pad(a, list(zip_longest([0], final-a.shape,
                                                 fillvalue=0)), 'constant')
                      for a in arrays])
print('out done')

out.shape

In [ ]:
sub_np = np.load(r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\activationMap\0_activation.npy')

In [ ]:
sub_np.shape

In [ ]:
print(type(out))
print(type(sub_np))

In [ ]:
out = torch.Tensor(out)

In [ ]:
sub_np = torch.from_numpy(np.load(r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\activationMap\0_activation.npy'))
print(out.requires_grad)
print(sub_np.requires_grad)

In [ ]:
sub_np.requires_grad_(True)
out.requires_grad_(True)
print(out.requires_grad)
print(sub_np.requires_grad)

In [ ]:
sub_param = nn.Parameter(sub_np,requires_grad=True)
optimizer = optim.SGD([sub_param], lr=0.01)

for epoch in range(5):
    optimizer.zero_grad()  # 그래디언트 초기화

    loss = loss_criterion(out,sub_param)
    print(loss)
    #loss.requires_grad_(True)
    # 역전파 및 업데이트
    loss.backward()
    optimizer.step()  # 활성화 맵 업데이트

    print(f"{epoch} : Updated input activation map:")
    #print(sub_param.data)

In [ ]:
# 예시로 사용할 텐서를 생성합니다.
sub_np = torch.randn((1, 10), requires_grad=True)  # 예시로 1x10 텐서 생성
print(type(sub_np))
print(sub_np.requires_grad)
# 텐서를 다른 파라미터로 감싸고 최적화합니다.
sub_param = nn.Parameter(sub_np)
optimizer = optim.SGD([sub_param], lr=0.1)

# 최적화를 위해 몇 번의 반복을 수행합니다.
for epoch in range(5):
    optimizer.zero_grad()  # 그래디언트 초기화

    # 손실 계산
    loss = torch.sum(sub_param ** 2)  # 예시로 간단한 손실 계산
    print(loss)

    # 역전파 및 업데이트
    loss.backward()
    optimizer.step()  # 파라미터 업데이트

    # 업데이트된 값을 출력합니다.
    print("Updated parameter value:")
    print(sub_param.data)  # 업데이트된 값 출력

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 예시로 사용할 텐서를 생성합니다.
sub_np = torch.from_numpy(np.load(r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\test_map\0\0_conv1.npy'))

# 텐서를 다른 파라미터로 감싸고 최적화합니다.
sub_param = nn.Parameter(sub_np)
optimizer = optim.Adam([sub_param], lr=0.1)

# 손실 함수 객체 생성
loss_criterion = nn.MSELoss()

# 최적화를 위해 몇 번의 반복을 수행합니다.
for epoch in range(5):
    optimizer.zero_grad()  # 그래디언트 초기화

    # 손실 계산
    loss = loss_criterion(n1, sub_param)  # MSE 손실 계산

    # 역전파 및 업데이트
    loss.backward()
    optimizer.step()  # 파라미터 업데이트

    # 업데이트된 값을 출력합니다.
    print("Updated parameter value:")
    print(sub_param.data)  # 업데이트된 값 출력

In [ ]:
path = r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\test_map\0'

In [ ]:
for c in os.listdir(path):
    print(c)
    break

In [ ]:
n1 = torch.from_numpy(np.load(r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\test_map\0\0_conv1.npy'))

In [ ]:
n1 = torch.flatten(n1)

In [ ]:
n1

In [ ]:
n1 = np.load(r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\test_map\0\0_conv1.npy')

In [ ]:
n1

In [ ]:
for a in activation.items():
    print(a[0])

In [ ]:
def act_iter(activation):
    for a in activation.items():
        yield a

In [ ]:
temp= act_iter(activation)

In [ ]:
for s in act_iter(activation):
    s[0]

In [ ]:
a = [2]

In [ ]:
loss_criterion

In [ ]:
n1 = torch.Tensor(np.load(r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\activationMap\0\0_conv1.npy'))
n2 = torch.Tensor(np.load(r'C:\Users\ms9804\Desktop\capstone\5.capstoneTestVer1\past\0\0_conv1.npy'))

In [ ]:
n1

In [ ]:
n2

In [ ]:
loss = loss_criterion(n1,n2)

In [ ]:
print(loss)

In [ ]:
activation['conv1'].shape

In [ ]:
temp = torch.Tensor(torch.rand(2,64,112,112))

In [ ]:
temp.shape

In [ ]:
len(temp)

In [ ]:
temp.shape

In [ ]:
np.shape(temp)

In [ ]:
val = []

In [ ]:
val.append(torch.Tensor(torch.rand(1,330)))

In [ ]:
val

In [ ]:
val = torch.cat(val,dim=0)

In [ ]:
val

In [ ]:
val.shape

In [ ]:
labels = []

In [ ]:
labels.append(torch.Tensor([0,0,0,1]))

In [ ]:
labels

In [ ]:
labels = torch.cat(labels)

In [ ]:
labels

In [ ]:
for l in labels:
    l = str(l.tolist())
    print(str(l))
    print(type(l))

In [ ]:
labels = torch.flatten(labels)

In [ ]:
labels

In [ ]:
temp = torch.flatten(torch.Tensor([0,1,2,1]))

In [ ]:
accuracy = accuracy_score(temp,labels)

In [ ]:
accuracy

In [ ]:
label = labels.tolist()

In [ ]:
str(label[0])

In [ ]:
type(str(label[0]))

In [ ]:
activation['conv1'].shape

In [ ]:
temp = []

In [ ]:
temp.append(activation['conv1'])

In [ ]:
import sys

In [ ]:
act = torch.randint(0,100,(2,))

In [ ]:
act

In [ ]:
act = act.tolist()

In [ ]:
act

In [ ]:
for a in act:
    print(type(a))

In [ ]:
def img_trans(aset = False):
    if aset == False:
        print('false')
    if aset == True:
        print('true')

In [ ]:
img_trans(aset = True)